<a href="https://colab.research.google.com/github/juhumkwon/YOLO/blob/main/yolo(%EC%9D%B4%EB%AF%B8%EC%A7%80%2C%20%EB%8F%99%EC%98%81%EC%83%81%20%EC%B6%94%EB%A1%A0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
file_path = '/content/drive/My Drive'
print(os.listdir(file_path))


Mounted at /content/drive
['Colab Notebooks', 'Data', '20241115_171806.jpg', '20241115_171800.jpg', '20241115_140203.jpg', '20241112_102319.jpg', '20241112_102259.jpg', '20241112_090619.jpg', '20241111_161500.jpg', '20241118_094841.jpg', '20241118_094831.jpg', '1731241210669-10.jpg', '1731241210669-9.jpg', '1731241210669-7.jpg', '1731241210669-6.jpg', '1731241210669-5.jpg', '1731241210669-4.jpg', '1731241210669-2.jpg', '1731241210669-1.jpg', '1731241210669-0.jpg', 'bcc81cd5847d441a677447774c72869d6b80a8d4aad29d6b014f66d68462b423.jpg', '4c22476737e90f67f16ec48fd3734708b95f700a359c4c6f1fd103c53fba49ab.jpg', 'ab097390cafe39a0a56a760fc726719676d117e909dfebeca172b0a052be4226.jpg', '9d1994da022791ce36f69e2d8dd1cddc88385fb2c64690e91deb2197ca3c2c3a.jpg', 'fa4b1daaa4fac6aacbbbf1033bd5eefaaa22f787b8d4b6daff99745f3664281e.jpg', '380e8d5db0dac8d628dc4c21266b0565457e1693fd7fccab3c140c284398ac13.jpg', 'cad915491972c69991669f3f82201c17dc14b469e3b2bba9ed0cc6a6b18c3c08.jpg', 'f2ddbe3a2198060f6cab6bf4df

In [ ]:
!pip install torch torchvision numpy pandas opencv-python
!git clone https://github.com/ultralytics/yolov5.git
!cd yolov5
!pip install -r requirements.txt


In [ ]:
# YOLOv5 전이 학습

import os
from pathlib import Path

# YOLOv5 라이브러리 가져오기
from yolov5 import train

# 커스텀 데이터셋 경로
DATASET_YAML = "path/to/your/dataset.yaml"  # YAML 파일 경로 (예: 'data/coco128.yaml')
PRETRAINED_MODEL = "yolov5s.pt"  # 사전 학습된 모델 (yolov5s, yolov5m 등)

# 저장 경로 설정
SAVE_DIR = "runs/train_custom"  # 학습 결과 저장 경로

# 학습 파라미터 설정
EPOCHS = 50  # 학습 반복 횟수
BATCH_SIZE = 16  # 배치 크기
IMG_SIZE = 640  # 입력 이미지 크기

def main():
    # YOLOv5 훈련 실행
    train.run(
        data=DATASET_YAML,
        weights=PRETRAINED_MODEL,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        img_size=IMG_SIZE,
        project=SAVE_DIR,
        name="exp",
    )

if __name__ == "__main__":
    main()


In [ ]:
# 데이터셋 준비
# 데이터셋은 YOLO 포맷(이미지와 라벨 파일)이어야 하며, dataset.yaml 파일로 정의해야 합니다.

train: ../data/train/images  # 학습 이미지 경로
val: ../data/val/images      # 검증 이미지 경로

# 클래스 이름 (예: 2개 클래스인 경우)
names:
  0: cat
  1: dog


In [ ]:
# 추론 코드
# YOLOv5로 학습된 모델을 이용하여 추론하는 간단한 예제 프로그램을 아래에 제공합니다.
# 이 코드는 학습된 모델(best.pt)을 사용하여 이미지나 동영상에서 객체를 감지합니다.

import torch
from pathlib import Path
import cv2
import matplotlib.pyplot as plt

# 학습된 모델 경로
MODEL_PATH = "runs/train_custom/exp/weights/best.pt"  # 학습된 모델 파일 경로
IMAGE_PATH = "path/to/your/image.jpg"  # 추론할 이미지 경로
VIDEO_PATH = "path/to/your/video.mp4"  # 추론할 동영상 경로 (옵션)

# 모델 로드
model = torch.hub.load('ultralytics/yolov5', 'custom', path=MODEL_PATH)  # 사용자 정의 모델 로드

def infer_image(image_path):
    # 이미지 로드 및 추론
    results = model(image_path)

    # 추론 결과 시각화
    results.render()  # 이미지 위에 박스와 라벨 그리기
    for img in results.imgs:
        plt.imshow(img[..., ::-1])  # BGR -> RGB로 변환
        plt.axis("off")
        plt.show()

def infer_video(video_path, output_path="output.mp4"):
    # 동영상 로드
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # MP4 코덱
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 추론 수행
        results = model(frame)

        # 추론 결과 이미지 위에 그리기
        for box in results.xyxy[0]:
            x1, y1, x2, y2, conf, cls = map(int, box[:6])
            label = f"{model.names[cls]} {conf:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        out.write(frame)  # 결과 저장
        cv2.imshow("YOLOv5 Inference", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # 이미지 추론
    infer_image(IMAGE_PATH)

    # 동영상 추론 (옵션)
    # infer_video(VIDEO_PATH, output_path="output_inference.mp4")


In [ ]:
import torch
import cv2
import matplotlib.pyplot as plt
from PIL import Image

# 모델 경로와 이미지 경로 설정
MODEL_PATH = "/content/drive/My Drive/yolov5/runs/train/yolov5s_results/weights/best.pt"
IMAGE_PATH = "/content/drive/My Drive/Data/yt-wfBfQzHF5K0-0027_jpg.rf.11f46c5dedd8c93d42ed4e03c80939c8.jpg"

# 모델 로드
model = torch.hub.load('ultralytics/yolov5', 'custom', path=MODEL_PATH)
model.conf = 0.1  # 신뢰도 임계값 설정

def infer_image(image_path):
    # 이미지 로드 및 RGB 변환
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR -> RGB 변환

    # 추론 실행
    results = model(image)

    # 감지된 객체 출력 (Pandas 형식)
    print("Detection results:")
    print(results.pandas().xyxy[0])  # 감지된 객체 정보 출력

    # 렌더링된 이미지 확인
    print("Rendered image available:", len(results.ims) > 0)  # 렌더링된 이미지 확인

    # 추론 결과 시각화
    results.render()  # 렌더링된 이미지에 바운딩 박스 그리기
    plt.figure(figsize=(10, 10))

    # BGR -> RGB 변환 후 이미지 표시
    plt.imshow(results.ims[0][..., ::-1])  # BGR -> RGB 변환
    plt.axis('off')
    plt.title("Detection Results: Burung Murai Batu")
    plt.show()

    # 렌더링된 이미지를 PIL 형식으로 변환 후 저장
    result_image = Image.fromarray(results.ims[0])  # 렌더링된 이미지를 PIL 형식으로 변환
    result_image.save("output.jpg")  # 결과 이미지를 파일로 저장
    print("결과 이미지가 'output.jpg'로 저장되었습니다.")

if __name__ == "__main__":
    infer_image(IMAGE_PATH)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-1-25 Python-3.11.11 torch-2.5.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Detection results:
         xmin        ymin        xmax        ymax  confidence  class  \
0  264.794006  160.803864  456.919556  401.781219    0.583673      3   

                name  
0  Burung Murai Batu  
Rendered image available: True
결과 이미지가 'output.jpg'로 저장되었습니다.


In [ ]:
import os

file_path = "/content/drive/My Drive/Data/bird.jpg"

if os.path.exists(file_path):
    print(f"파일이 존재합니다: {file_path}")
else:
    print(f"파일이 존재하지 않습니다: {file_path}")


파일이 존재합니다: /content/drive/My Drive/Data/bird.jpg
